In [1]:
import json
import argparse

from utils import invoke, evaluate_time_predictions
from threadpool import ThreadPool
from tqdm import tqdm

prompt_file = "prompts/v4.txt"
bench_file = "../eval_ppl/dataset/multi_time_queries.json"

with open(prompt_file) as f:
    prompt = f.read()

with open(bench_file) as f:
    time_bench = json.load(f)

# Function to process a single example
def process_example(example, prompt):
    # params = {
    #     "question": example["question"],
    #     "current_time_iso": example["now"],
    #     "date_field": "timestamp",
    #     "other_date_fields": []
    # }
    params = {
        "question": example["question"],
        "current_time_iso": example["now"][11:-2],
        "date_field": example["date_field"],
        "other_date_fields": example["other_date_fields"]
    }
    
    return invoke(prompt, **params)


# Create a thread pool
thread_pool = ThreadPool(max_workers=2)

# Use the thread pool to process all examples in parallel
predicts = thread_pool.map_with_args(
    func=process_example,
    items=time_bench[2:4],
    fixed_args={"prompt": prompt},
    desc="Processing examples",
)

print(predicts)

Processing examples:   0%|          | 0/2 [00:00<?, ?it/s]

Processing examples: 100%|██████████| 2/2 [00:02<00:00,  1.14s/it]

[{'analysis': '1. Time expressions identified: ["in the last 6 month", "in the last 9 years"]\n2. After filtering aggregation references: ["in the last 6 month", "in the last 9 years"]\n3. Semantic context analysis:\n   - "in the last 6 month" modifies "orders" → maps to order_date\n   - "in the last 9 years" modifies "products created" → maps to products.created_on\n4. order_date expressions: ["in the last 6 month"]\n5. Time range calculation: \n   - "in the last 6 month" for 2025-12-20 03:53:30 → 2025-06-20 03:53:30 to 2025-12-20 03:53:30', 'start': '2025-06-20 03:53:30', 'end': '2025-12-20 03:53:30', 'query': 'Count the number of orders that contain products created in the last 9 years.'}, {'analysis': '1. Time expressions identified: ["in the last 9 years"]\n2. After filtering aggregation references: ["in the last 9 years"]\n3. Semantic context analysis:\n   - "in the last 9 years" modifies "products created" → maps to products.created_on\n4. order_date expressions: []\n5. Time ran

In [2]:
for i in range(2):
    for key in ["start", "end"]:
        if predicts[i][key]!=None:
            predicts[i][key] = predicts[i][key].replace("T", " ")
    
    print(time_bench[i]["now"], time_bench[i]["question"])
    print(predicts[i]["start"], predicts[i]["end"])
    print(predicts[i]["query"])
    print()
    print(predicts[i]["analysis"])
    print("*"*100)

TIMESTAMP('2025-05-27 03:53:30') Show me all orders from last week with products created after 2016-12-20
2025-06-20 03:53:30 2025-12-20 03:53:30
Count the number of orders that contain products created in the last 9 years.

1. Time expressions identified: ["in the last 6 month", "in the last 9 years"]
2. After filtering aggregation references: ["in the last 6 month", "in the last 9 years"]
3. Semantic context analysis:
   - "in the last 6 month" modifies "orders" → maps to order_date
   - "in the last 9 years" modifies "products created" → maps to products.created_on
4. order_date expressions: ["in the last 6 month"]
5. Time range calculation: 
   - "in the last 6 month" for 2025-12-20 03:53:30 → 2025-06-20 03:53:30 to 2025-12-20 03:53:30
****************************************************************************************************
TIMESTAMP('2025-07-27 03:53:30') Show me all orders with products created after 2016-12-20
None None
Count the number of orders that contain products